In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Carga y preprocesamiento

In [ ]:
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder


CPU = False

###Lee el dataset desde un directorio cuyos subdirectorios separan las diferentes
### clases que contiene. Devuelve un diccionario
#La opción merge_similar toma como la misma todas las clases cuyo nombre empiece 
# por la misma palabra
#El parámetro encoder en un LabelEncoder de sklearn. Si no se le pasa, se crea uno
# nuevo. Pasarlo como parámetro puede servir para descodificar las etiquetas
#El parámetro transform_fnc sirve para aplicar el preprocesado por cada chunk de 
# imágenes para evitar el costo de memoria de transformar todas a la vez
def load_dataset(directory_path, shuffle=True, merge_similar=False, cribe=1):
    from sklearn.preprocessing import LabelEncoder
    import glob, os
    
    current_dir = os.path.abspath(os.getcwd())
    os.chdir(directory_path)
    imgs, labels = [], []
    
    for class_dir in os.listdir('./'):
        print("Class:",class_dir)
        class_name = class_dir if not merge_similar else class_dir.split()[0]
        class_imgs = []
        for i, file in enumerate(glob.glob("{}/*.jpg".format(class_dir))):
            if i % cribe == 0:
                imgs.append(cv.imread(file))
                labels.append(class_name)
        
    imgs = np.asarray(imgs)
    labels = np.asarray(labels)
    if shuffle:
        indices = np.random.permutation(len(imgs))
        imgs = imgs[indices]
        labels = labels[indices]
            
    os.chdir(current_dir)
    return imgs, labels

In [ ]:
x_train, y_train = load_dataset('./fruits360reduc/Training/', cribe=4)

In [ ]:
x_test, y_test = load_dataset('./fruits360reduc/Test/', cribe=1)

In [ ]:
x_train = np.asarray( [cv.resize(img, (64,64)) for img in x_train], np.float32 )
x_test = np.asarray( [cv.resize(img, (64,64)) for img in x_test], np.float32 )

In [ ]:
mu = np.mean(x_train, axis=(0,1,2))
sigma = np.std(x_train, axis=(0,1,2))

In [ ]:
x_train -= mu
x_train /= sigma

In [ ]:
x_test -= mu
x_test /= sigma

In [ ]:
x_train = x_train.transpose((0,3,1,2))
x_test = x_test.transpose((0,3,1,2))

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
from sklearn.model_selection import train_test_split

batch_size = 128
x_train, x_val, y_train, y_val = train_test_split(x_train, 
                                                  y_train, 
                                                  test_size=0.15, 
                                                  shuffle=True, 
                                                  random_state=57)

### Caffe

In [ ]:
if CPU:
    !sudo apt install caffe-cpu
else:
    !sudo apt install caffe-cuda

In [ ]:
import caffe

if not CPU:
    caffe.set_mode_gpu()
    caffe.set_device(0)

In [ ]:
def step_over_chunks(solver, data_chunks, labels_chunks, img_shape):
    for dchunk, lchunk in zip(data_chunks, labels_chunks):
        new_shape = (len(dchunk),) + img_shape
        if solver.net.blobs['data'].data.shape != new_shape:
            solver.net.blobs['data'].reshape(*new_shape)
            solver.net.blobs['label'].reshape(len(dchunk))
        for i in range(len(dchunk)):
            solver.net.blobs['data'].data[i] = dchunk[i]
            solver.net.blobs['label'].data[i] = lchunk[i]
        solver.step(1)

def test_net(test_net, x_test, y_test, batch_size):
    test_data_chunks = [ x_test[x:x+batch_size] for x in range(0, len(x_test), batch_size) ]
    test_labels_chunks = [ np.squeeze(y_test[y:y+batch_size]) for y in range(0, len(y_test), batch_size) ]
    loss_sum = 0
    accuracy = 0

    for dchunk, lchunk in zip(test_data_chunks, test_labels_chunks):
        new_shape = (len(dchunk),) + x_test[0].shape
        if test_net.blobs['data'].data.shape != new_shape:
            test_net.blobs['data'].reshape(*new_shape)
            test_net.blobs['label'].reshape(len(dchunk))
        test_net.blobs['data'].data[:] = dchunk[:]
        test_net.blobs['label'].data[:] = lchunk[:]

        test_net.forward()
        loss_sum += test_net.blobs['loss'].data
        accuracy += test_net.blobs['accuracy'].data

    return loss_sum/len(test_data_chunks), accuracy/len(test_data_chunks)

def fit(solver, x_train, y_train, x_val, y_val, niter, batch_size, patience=5, epsilon=0.0001):
    train_loss, val_loss = np.zeros(niter), np.zeros(niter)

    data_chunks = [ x_train[x:x+batch_size] for x in range(0, len(x_train), batch_size) ]
    labels_chunks = [ y_train[y:y+batch_size] for y in range(0, len(y_train), batch_size) ]

    final_it = niter-1
    best_loss = float("inf")
    worse_loss_counter = 0

    for it in range(niter):
        step_over_chunks(solver, data_chunks, labels_chunks, x_train[0].shape)
        train_loss[it] = solver.net.blobs['loss'].data
        #train_acc[it] = solver.net.blobs['accuracy'].data
        val_loss[it], _ = test_net(solver.test_nets[0], x_val, y_val, batch_size)
        print('iter {}, loss={}, val_loss={}'.format(it, train_loss[it], val_loss[it]))

        if val_loss[it] < best_loss - epsilon:
            best_loss = val_loss[it]
            worse_loss_counter = 0
        else:
            worse_loss_counter += 1
            if worse_loss_counter > patience:
                final_it = it
                break
    
    return train_loss[0:final_it+1], val_loss[0:final_it+1]

In [ ]:
# Entrenamiento última capa

solver_fruits360 = caffe.get_solver('./model_fruits360/solver_fruits360.prototxt')
solver_fruits360.net.copy_from('./model_cfn_jps/cfn_jps.caffemodel')

train_loss, val_loss = fit(solver_fruits360, x_train, y_train, x_val, y_val, 200, batch_size)

In [ ]:
# Comprobar la precisión del modelo final

_, accuracy = test_net(solver_fruits360.test_nets[0], x_test, y_test, batch_size)
print("Precisión en test: {}".format(accuracy))

Precisión en test: 0.9119318181818182


In [ ]:
# Guardar los pesos del modelo

solver_fruits360.net.save('./model_fruits360/fruits360.caffemodel')

In [ ]:
# Gráfica train-loss, val-loss

_, ax1 = plt.subplots()
ax1.plot(np.arange(len(train_loss)), train_loss, label='Train loss')
ax1.plot(np.arange(len(val_loss)), val_loss, label='Validation loss')
ax1.set_xlabel('Epochs')
ax1.legend(loc='best')

In [ ]:
# Fine Tuning

solver_fruits360_ft = caffe.get_solver('./model_fruits360/solver_fruits360_fine_tuning.prototxt')
solver_fruits360_ft.net.copy_from('./model_fruits360/fruits360.caffemodel')

train_loss_ft, val_loss_ft = fit(solver_fruits360_ft, x_train, y_train, x_val, y_val, 50, batch_size, patience=1)

In [ ]:
# Comprobar la precisión del modelo final

_, accuracy = test_net(solver_fruits360_ft.test_nets[0], x_test, y_test, batch_size)
print("Precisión en test: {}".format(accuracy))

In [ ]:
# Guardar los pesos del modelo

solver_fruits360_ft.net.save('./model_fruits360/fruits360_fine_tuning.caffemodel')

In [ ]:
# Gráfica train-loss, val-loss

_, ax1 = plt.subplots()
ax1.plot(np.arange(len(train_loss_ft)), train_loss_ft, label='Train loss')
ax1.plot(np.arange(len(val_loss_ft)), val_loss_ft, label='Validation loss')
ax1.set_xlabel('Epochs')
ax1.legend(loc='best')